<a href="https://colab.research.google.com/github/dtabuena/Workshop/blob/main/Zalocusky_Data_Scanpy_MERGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This file is stored as a colab notebook but must be run locally due to the size of gene data files.

In [1]:
!pip install scanpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82813 sha256=c7b1bed3cf44dccc971d7203c5b59aa53d3a080a05db63adb943da4e88c8af12
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8026 sha256=6220c846d2295e416a6f0a84e346e8875bc023564f8ceced073d5283bf8ad067
  Stored in directory: /root/.cache/pip/wheels/6a/aa/

In [2]:
import h5py
import numpy as np
import scipy as sci
from matplotlib import pyplot as plt
import scanpy as sc
import tarfile
import os
import anndata as ad
import pandas as pd


sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.9.3 anndata==0.9.2 umap==0.5.3 numpy==1.22.4 scipy==1.10.1 pandas==1.5.3 scikit-learn==1.2.2 statsmodels==0.13.5 pynndescent==0.5.10


In [3]:
def trim_key(k):
    k = k.replace('_raw_gene_bc_matrices_h5.h5',"")
    k = k.replace('_raw_gene_bc_matrics_h5.h5',"")
    k =k.replace(k.split("_")[0],"")
    return k

In [4]:
path = 'C:/Users/dennis.tabuena/Downloads/GSE167497_RAW (2).tar'
my_tar = tarfile.open(path)
my_tar.extractall('./animal_results') # specify which folder to extract to
my_tar.close()
for f in os.listdir('./animal_results'):
    print(f)

FileNotFoundError: ignored

In [ ]:
adata_dict = {}
for f in os.listdir('./animal_results'):
    if 'YH' not in f:
        a = sc.read_10x_h5('./animal_results/'+f)
        a.var_names_make_unique()
        adata_dict[trim_key(f)] = a

C:\Users\dennis.tabuena\AppData\Local\anaconda3\lib\site-packages\anndata\_core\anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
C:\Users\dennis.tabuena\AppData\Local\anaconda3\lib\site-packages\anndata\_core\anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
C:\Users\dennis.tabuena\AppData\Local\anaconda3\lib\site-packages\anndata\_core\anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
C:\Users\dennis.tabuena\AppData\Local\anaconda3\lib\site-packages\anndata\_core\anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
C:\Users\dennis.tabuena\AppData\Local\anaconda3\lib\site-packages\anndata\_core\

In [ ]:
adata = ad.concat(adata_dict,axis = 0,label="Sample",index_unique="_")
display(adata.obs.head())


,Sample
AAACCTGAGAAACCAT-1__02_E4_14_H0,_02_E4_14_H0
AAACCTGAGAAACCGC-1__02_E4_14_H0,_02_E4_14_H0
AAACCTGAGAAACCTA-1__02_E4_14_H0,_02_E4_14_H0
AAACCTGAGAAACGAG-1__02_E4_14_H0,_02_E4_14_H0
AAACCTGAGAAACGCC-1__02_E4_14_H0,_02_E4_14_H0


In [ ]:
adata_filt = adata.copy()

adata_filt.var['mt'] = adata_filt.var_names.str.startswith('mt-')
sc.pp.calculate_qc_metrics(adata_filt, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
adata_filt = adata_filt[adata_filt.obs.pct_counts_mt < 0.25, :]
sc.pp.filter_cells(adata_filt, min_genes=475)
sc.pp.filter_cells(adata_filt, max_genes=4000)
sc.pp.filter_genes(adata_filt, min_cells=1)
sc.pp.filter_genes(adata_filt, max_cells=np.inf)
display(adata_filt)


View of AnnData object with n_obs × n_vars = 14297602 × 27998
    obs: 'Sample', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

filtered out 14159293 cells that have less than 475 genes expressed


C:\Users\dennis.tabuena\AppData\Local\anaconda3\lib\site-packages\scanpy\preprocessing\_simple.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number


filtered out 168 cells that have more than 4000 genes expressed
filtered out 4470 genes that are detected in less than 1 cells


AnnData object with n_obs × n_vars = 138141 × 23528
    obs: 'Sample', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells'